In [2]:
import pandas as pd
import mysql.connector as mysqlpy

In [3]:
#ouverture du fichier txt
txt_content = open("country.txt", "r")

#modification du fichier txt sous la forme SQL
txt = ''
for row in txt_content:
    k = row.replace("'", '.')
    j = k.replace(',', '')
    i = j.replace(' __****__ ', "', '")
    txt = txt + "('" + i + "'), "

une_ligne = txt.replace('\n', '')
values = une_ligne[:-2]

In [4]:
#ouverture du fichier csv
data = pd.read_csv('population.csv')

#Changement du format des dates
for i in range(data.shape[0]):
    ligne = str(data['TIME'][i])
    ligne = ligne + "-01-01"
    data.loc[[i], ['TIME']] = ligne

In [10]:
user = 'root'
password = 'example'
host = 'localhost'
port = '3307'
database = 'CountriesDatas'

bdd = mysqlpy.connect(user=user, password=password, host=host, port=port, database=database)
cursor = bdd.cursor()

#QUESTION III
def fill_countries():
    """Remplissage de la table countries"""
    query = f"INSERT IGNORE INTO countries(code, name) VALUES {values};"
    cursor.execute(query)

def fill_population():
    """Remplissage de la table populations"""
    for i in range(data.shape[0]):
        id = i + 1
        location = data['LOCATION'][i]
        time = data['TIME'][i]
        value = data['Value'][i]
        query = f"""INSERT INTO populations(id, LOCATION, TIME, Value) 
        VALUES ('{id}', '{location}', '{time}', {value})
        ON DUPLICATE KEY UPDATE id={id}"""
        cursor.execute(query)

#QUESTION IV
def affiche_infos():
    """Retourne une liste de tuples contenants les infos sur la population des pays"""
    populations_infos = []
    query = """SELECT populations.id, countries.name, populations.TIME, populations.Value 
    FROM populations JOIN countries 
    WHERE populations.LOCATION = countries.code"""
    cursor.execute(query)
    for pop_row in cursor:
        populations_infos.append(pop_row)
    return populations_infos

#QUESTION V
df = pd.DataFrame(data=affiche_infos(), columns=['id', 'country_name', 'date', 'population_value'])
print(df)

#fill_countries()
#fill_population()
#print(affiche_infos())

bdd.commit()
cursor.close()
bdd.close()

          id country_name        date  population_value
0          1    Australia  1951-01-01          2.971130
1          2    Australia  1952-01-01          2.550550
2          3    Australia  1953-01-01          2.070280
3          4    Australia  1954-01-01          1.942080
4          5    Australia  1955-01-01          2.372450
...      ...          ...         ...               ...
12332  15628    Singapore  2016-01-01          0.790967
12333  15629    Singapore  2017-01-01          0.819538
12334  15630    Singapore  2018-01-01          0.718317
12335  15631    Singapore  2019-01-01          0.799292
12336  15632    Singapore  2020-01-01          0.447096

[12337 rows x 4 columns]
